# Фильтрация документов


<p> В этой главе мы покажем, как можно классифицировать документы по их содержимому.
Это очень полезное применение искусственного интеллекта получает все большее распространение, и в первую 
очередь для фильтрации спама. Из-за легкодоступности и исключительной дешевизны отправки сообщений по электронной почте 
возникла серьезная проблема – любой человек, чей адрес попал в нечистоплотные руки, начинает получать непрошенную почту, а это 
затрудняет выделение действительных важных писем.</p>


<p> Разумеется, проблеме спама подвержена не только электронная почта.
Веб-сайты, ставшие со временем более интерактивными, предлагают
посетителям оставлять свои комментарии и даже создавать оригинальный контент. И это тоже оставляет возможность для спама. Публичные форумы, например Yahoo! Groups и Usenet, уже давно стали мишенями для размещения не относящихся к теме форума сообщений или
рекламы сомнительной продукции. Теперь от той же напасти страдают
сайты Википедии и блоги. При создании приложений, разрешающих
что-то публиковать любому желающему, необходимо заранее подумать
о том, как бороться со спамом.
Описанные в этой главе алгоритмы применимы не только для фильтрации спама. Они решают общую задачу обучения распознаванию того,
принадлежит ли документ к той или иной категории, а стало быть, могут применяться и для не столь дурно пахнущих целей. Например,
можно автоматически рассортировать поступающую в ваш ящик почту на письма, относящиеся к работе и к личной жизни. Или распознавать письма с запросом информации и автоматически направлять их
лицу, способному дать наиболее компетентный ответ. В конце главы
мы приведем пример автоматической классификации сообщений из
RSS-канала.</p>


## Фильтрация спама
Первые попытки фильтрации спама полагались на классификаторы на
основе правил, когда человек сам формулировал правила, в соответствии с которыми сообщение признавалось полезным или спамом.
Обычно правила касались слишком большого числа заглавных букв,
упоминания фармацевтической продукции или чрезмерно пестрой
раскраски. Но довольно быстро выявился очевидный недостаток подобных классификаторов – спамеры изучили все правила и перестали
явно выдавать себя, научившись обходить фильтры. И наоборот, люди,
чьи престарелые родители так и не научились нажимать клавишу Caps
Lock, обнаружили, что вполне нормальные сообщения от них классифицируются как спам.
У фильтров на основе правил есть и еще один минус. Признаки спама
меняются в зависимости от того, откуда отправлено сообщение и кому
оно адресовано. Ключевые слова, которые для одного пользователя,
форума или раздела Википедии служат несомненным признаком спама, в другой ситуации считаются совершенно нормальными. Чтобы
разрешить эту проблему, мы в данной главе рассмотрим программы,
которые обучаются по мере того, как вы сообщаете им, что считать
спамом, а что нет, причем это происходит как на начальной стадии,
так и в процессе получения новых сообщений. Таким образом можно
подготовить различные экземпляры и наборы данных для разных
пользователей, групп и сайтов, каждый из которых будет по-своему
уточнять, что такое спам.

## Документы и слова
Классификатору, который мы построим, будут необходимы признаки
для классификации различных образцов. Признаком можно считать
любое свойство, относительно которого можно сказать, присутствует
оно в образце или нет. Если классифицируются документы, то образцом считается документ, а признаками – встречающиеся в нем слова.
Когда слова рассматриваются как признаки, мы предполагаем, что некоторые слова вероятнее встретить в спаме, чем в нормальных сообщениях. Именно это допущение лежит в основе большинство антиспамных фильтров. Но никто не заставляет ограничиваться только словами. В качестве признаков можно рассматривать пары слов, целые фразы и вообще все, о чем можно сказать, присутствует оно в документе
или отсутствует.
Создадим функцию <b>getwords</b>,
которая будет извлекать признаки из текста:

In [40]:
import re
import math
def getwords(doc):
    splitter=re.compile('\s')
    # Split the words by non-alpha characters
    words=[s.lower() for s in splitter.split(doc) if len(s)>2 and len(s)<20] 
    # Return the unique set of words only
    return dict([(w,1) for w in words])

<p>Эта функция разбивает текст на слова по любому символу, отличномуот буквы. Она оставляет только сами слова, преобразованные в нижний регистр. Решить, что будет считаться признаками, – это самая сложная и самая важная задача. Признаки должны быть достаточно распространенными, чтобы встречаться часто, но не настолько распространенными, чтобы попадаться в каждом документе. Теоретически признаком может быть весь текст документа, но это почти всегда бессмысленно, разве что вы получаете одно и то же сообщение снова и снова. Другая крайность – объявить признаками отдельные символы. Но, поскольку они входят в каждое сообщение, то не позволят отделить зерна от плевел. Даже при выборе слов в качестве признаков возникает ряд вопросов: как выделять слова, какие знаки препинания в них включать и следует ли учитывать информацию в заголовке.</p>
<p>Еще один фактор, который нужно принимать во внимание, когда выбирается набор признаков, – насколько хорошо они смогут распределить документы по категориям. Например, приведенная выше функция getwords сокращает число признаков, приводя все слова к нижнему регистру. Это означает, что написанное заглавными буквами слово, встречающееся в начале предложения, ничем не отличается от такого же слова, написанного строчными буквами, в середине предложения. Но тем самым мы полностью игнорируем КРИКЛИВЫЙ стиль, характерный для многих спамных сообщений, а это мог бы быть важный классифицирующий признак спама. Альтернативной могло быть стать объявление признаком такой характеристики: более половины слов написаны заглавными буквами. 
<p>Как видите, при выборе признаков приходится идти на различные компромиссы и постоянно уточнять ранее принятые решения. Пока
    мы будем пользоваться простой функцией <b>getwords</b>, а в конце главы предложим ряд идей по совершенствованию механизма выделения
признаков.</p>

## Обучение классификатора
<p> Рассматриваемые в этой главе классификаторы способны обучаться тому, как надо классифицировать документы. Многие алгоритмы, в частности нейронные сети, обучаются, когда им предъявляют примеры правильных ответов. Чем больше документов и правильных способов классификации видит алгоритм, тем лучше он классифицирует последующие документы. Кроме того, в начальный момент классификатор намеренно делают очень «робким», так чтобы его «уверенность в себе» повышалась по мере того, как он узнает, какие признаки важны для проведения различий.</p>
<p>Прежде всего нам понадобится класс для представления классификатора. Он инкапсулирует те факты, которым классификатор успел обучиться. У такой структуры есть важное достоинство – можно создавать
разные экземпляры классификаторов, ориентированные на различных пользователей, группы или запросы, и обучать их в соответствии
с потребностями конкретной группы. Создадим следующий класс <b>classifier</b>:</p>


In [41]:
class classifier:
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        self.fc={}
        # Counts of documents in each category
        self.cc={}
        self.getfeatures=getfeatures

<p>В этом классе есть три переменных экземпляра:<i> fc, cc и getfeatures.</i> В переменной <i>fc</i> хранятся счетчики признаков при различных классификациях: например:
    <i>{'python': {'bad': 0, 'good': 6}, 'the': {'bad': 3, 'good': 3}}</i>
Это означает, что слово the трижды появлялось в документах, классифицированных как плохие, и трижды – в документах, классифицированных как хорошие. Слово Python появлялось только в хороших документах.</p>
<p>Переменная <i>cc</i> – это словарь, в котором отражено, сколько раз применялась каждая классификация. Это необходимо для вычисления вероятности, о чем чуть позже. И последняя переменная экземпляра, <i>getfeatures</i>, – это функция, с помощью которой из классифицируемых образцов выделяются признаки. В нашем примере это функция <b>getwords</b>.</p>
Методы класса не будут пользоваться этими словарями напрямую, поскольку это ограничило бы возможности для сохранения данных
обучения – в файле, в базе или еще где-то. Создадим следующие вспомогательные функции для увеличения и получения счетчиков:

In [42]:
class classifier:
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        self.fc={}
        # Counts of documents in each category
        self.cc={}
        self.getfeatures=getfeatures

    def incf(self,f,cat):  
        self.fc.setdefault(f,{})  
        self.fc[f].setdefault(cat,0)  
        self.fc[f][cat]+=1
        
    def incc(self,cat):  
        self.cc.setdefault(cat,0)  
        self.cc[cat]+=1
        
    def fcount(self,f,cat):  
        if (f in self.fc) and (cat in self.fc[f]):    
            return float(self.fc[f][cat])  
        return 0.0
    
    def catcount(self,cat):  
        if cat in self.cc:    
            return float(self.cc[cat])  
        return 0
    
    def totalcount(self):  
        return sum(self.cc.values( ))
    
    def categories(self):  
        return self.cc.keys( )
    
    def train(self,item,cat):  
        features=self.getfeatures(item)  
        # Увеличить счетчики для каждого признака в данной классификации
        for f in features:    
            self.incf(f,cat)  
            # Увеличить счетчик применений этой классификации  
        self.incc(cat)

Убедимся, что класс работает правильно

In [43]:
cl=classifier(getwords)
cl.train('the quick brown fox jumps over the lazy dog','good');
cl.train('make quick money in the online casino','bad');
cl.fcount('quick','good')

1.0

In [44]:
cl.fcount('quick','good')

1.0

Метод <i>train</i> принимает образец (в данном случае документ) и классификацию. С помощью функции getfeatures он выделяет из образца признаки. Затем он вызывает метод <i>incf</i>, чтобы увеличить счетчики для каждого признака в данной классификации. И наконец он увеличивает счетчик применений этой классификации:


Сейчас нам был бы полезен метод, который сохраняет в классификаторе некоторые данные, полученные в ходе обучения, чтобы не приходилось каждый раз проводить обучение заново. Добавим такую функцию:


In [45]:
def sampletrain(cl):
    cl.train('Nobody owns the water.','good')
    cl.train('the quick rabbit jumps fences','good')
    cl.train('buy pharmaceuticals now','bad')
    cl.train('make quick money at the online casino','bad')
    cl.train('the quick brown fox jumps','good')

## Вычисление вероятностей
Теперь у нас есть счетчики, показывающие, как сообщения были разнесены по категориям. Следующий шаг – превратить эти числа в вероятности. Вероятностью называется число от 0 до 1, показывающее частоту возникновения некоторого события. В данном случае мы вычисляем вероятность того, что слово принадлежит конкретной категории. Для этого нужно разделить количество вхождений данного слова в документ, отнесенный к этой категории, на общее число документов в той же категории.

Добавим в класс <i>classifier</i> метод <i>fprob</i>:

In [46]:
class classifier:
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        self.fc={}
        # Counts of documents in each category
        self.cc={}
        self.getfeatures=getfeatures

    def incf(self,f,cat):  
        self.fc.setdefault(f,{})  
        self.fc[f].setdefault(cat,0)  
        self.fc[f][cat]+=1
        
    def incc(self,cat):  
        self.cc.setdefault(cat,0)  
        self.cc[cat]+=1
        
    def fcount(self,f,cat):  
        if (f in self.fc) and (cat in self.fc[f]):    
            return float(self.fc[f][cat])  
        return 0.0
    
    def catcount(self,cat):  
        if cat in self.cc:    
            return float(self.cc[cat])  
        return 0
    
    def totalcount(self):  
        return sum(self.cc.values( ))
    
    def categories(self):  
        return self.cc.keys( )
    
    def train(self,item,cat):  
        features=self.getfeatures(item)  
        # Увеличить счетчики для каждого признака в данной классификации
        for f in features:    
            self.incf(f,cat)  
            # Увеличить счетчик применений этой классификации  
        self.incc(cat)
        
        
    def fprob(self,f,cat):  
        if self.catcount(cat)==0: return 0  
        # Общее число раз, когда данный признак появлялся в этой категории,  
        # делим на количество образцов в той же категории  
        return self.fcount(f,cat)/self.catcount(cat)


Эта величина называется условной вероятностью, обычно записывается в виде Pr(A | B) и произносится так: «Вероятность A при условии B». Мы только что вычислили значение Pr(Слово | Классификация); то есть вероятность того, что некоторое слово появитсz в данной классификации. Протестируем эту функцию:


In [47]:
cl=classifier(getwords)
sampletrain(cl)
cl.fprob('quick','good')

0.6666666666666666

### Начинаем с разумной гипотезы
Метод fprob дает точный результат для тех признаков и классифика-ций, которые он уже видел, однако имеется небольшая проблема – ис-пользование лишь той информации, с которой метод уже сталкивался, делает его слишком чувствительным на ранних этапах обучения и по отношению к редко встречающимся словам. В нашем примере слово money (деньги) встретилось только в одном документе, который клас-сифицирован как плохой, потому что содержит рекламу казино. Раз это слово встретилось в одном плохом документе и не встречалось в хо-роших, то вероятность того, что оно может появиться в хороших доку-ментах, согласно функции fprob равна 0. Это, пожалуй, перебор, так как слово money вполне нейтральное, просто ему не повезло – угораздило первый раз встретиться в плохом документе. Было бы разумнее, если бы вероятность постепенно приближалась к нулю по мере того, как об-рабатывается все больше и больше документов в той же категории. 

Чтобы справиться с этой проблемой, мы выберем некую предполагае-мую  вероятность,  которой  будем  пользоваться,  когда  информации о рассматриваемом признаке слишком мало. Хорошей отправной точ-кой может послужить вероятность 0,5. Еще нужно решить, какой вес приписать предполагаемой вероятности, – 1 означает, что вес предпо-лагаемой вероятности равен одному слову. Взвешенная вероятность – это средневзвешенное значение, возвращаемое функцией getprobability, и предполагаемая вероятность.

В рассмотренном выше примере взвешенная вероятность слова money первоначально равна 0,5 во всех категориях. После того как классификатору был предъявлен один плохой документ и он узнал, что слово money  отнесено  к  категории  плохих,  вероятность,  что  оно  окажется плохим, возросла до 0,75. Это значение дает следующее вычисление:

(weight*assumedprob + count*fprob)/(count+weight)

= (1 x 1.0+1 x 0.5) / (1.0 + 1.0)

= 0.75

Добавим метод weightedprob в класс classifier:

In [48]:
class classifier:
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        self.fc={}
        # Counts of documents in each category
        self.cc={}
        self.getfeatures=getfeatures

    def incf(self,f,cat):  
        self.fc.setdefault(f,{})  
        self.fc[f].setdefault(cat,0)  
        self.fc[f][cat]+=1
        
    def incc(self,cat):  
        self.cc.setdefault(cat,0)  
        self.cc[cat]+=1
        
    def fcount(self,f,cat):  
        if (f in self.fc) and (cat in self.fc[f]):    
            return float(self.fc[f][cat])  
        return 0.0
    
    def catcount(self,cat):  
        if cat in self.cc:    
            return float(self.cc[cat])  
        return 0
    
    def totalcount(self):  
        return sum(self.cc.values( ))
    
    def categories(self):  
        return self.cc.keys( )
    
    def train(self,item,cat):  
        features=self.getfeatures(item)  
        # Увеличить счетчики для каждого признака в данной классификации
        for f in features:    
            self.incf(f,cat)  
            # Увеличить счетчик применений этой классификации  
        self.incc(cat)
        
        
    def fprob(self,f,cat):  
        if self.catcount(cat)==0: return 0  
        # Общее число раз, когда данный признак появлялся в этой категории,  
        # делим на количество образцов в той же категории  
        return self.fcount(f,cat)/self.catcount(cat)    
    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        # Calculate current probability
        basicprob=prf(f,cat)

        # Count the number of times this feature has appeared in
        # all categories
        totals=sum([self.fcount(f,c) for c in self.categories()])

        # Calculate the weighted average
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp


Протестируйм эту функцию в интерактивном сеансе. Перезагрузите модуль и повторно выполните метод sampletrain, поскольку при создании нового экземпляра класса вся накопленная за время обучения информация стирается:

In [49]:
cl=classifier(getwords)
sampletrain(cl)
print(cl.weightedprob('money','good',cl.fprob))
sampletrain(cl)
print(cl.weightedprob('money','good',cl.fprob))

0.25
0.16666666666666666


Как видите, после повторного запуска sampletrain классификатор еще сильнее утвердился во мнении относительно вероятностей различных слов – из-за вклада предполагаемой вероятности.

Мы выбрали для предполагаемой вероятности значение 0,5 просто потому, что оно лежит посередине между 0 и 1. Однако не исключено, что у вас имеется более точная априорная информация даже для необученного классификатора. Например, человек, приступающий к обучению антиспамного фильтра, может позаимствовать вероятности у фильтров, обученных другими людьми. Он по-прежнему сможет настроить фильтр под себя, но при этом классификатор будет лучше обрабатывать редко встречающиеся слова.

# Наивная классификация
Имея вероятности для слов, входящих в документ, вы должны выбрать какой-то способ комбинирования вероятностей отдельных слов для
вычисления вероятности того, что документ в целом принадлежит данной категории. В этой главе мы рассмотрим два разных метода классификации. Оба работают в большинстве случаев, но несколько отличаются по качеству при решении конкретных задач. Предметом этого раздела будет наивный байесовский классификатор.

Слово «наивный» в данном контексте означает следующее: классификатор предполагает, что комбинируемые вероятности независимы друг
от друга. Иными словами, вероятность того, что одно слово в документе относится к некоторой категории, никак не связана с вероятностями отнесения к той же категории других слов. Это предположение неверно, так как шансы, что документы, содержащие слово casino, содержат также и слово money, гораздо выше, чем шансы встретить слово money в документах о программировании на языке Python.

Следовательно, из-за ложного допущения о независимости нельзя использовать вероятность, вычисленную наивным байесовским классификатором, как истинную вероятность того, что документ принадлежит к некоторой категории. Однако можно сравнить результаты, полученные для разных категорий, и посмотреть, для какой из них вероятность оказалась самой большой. На практике, несмотря на ложность сходного предположения, этот метод классификации документов казывается на удивление эффективным.

## Вероятность для всего документа
Чтобы воспользоваться наивным байесовским классификатором, нужно сначала вычислить вероятность для документа в целом в рамках
данной классификации. Поскольку мы предположили, что вероятности независимы, то для этого достаточно их перемножить.

Предположим, например, что слово Python встречается в 20% плохих документов, то есть

*Pr(Python | Плохой) = 0,2,*

а слово casino – в 80% плохих документов:

*Pr(Casino | Плохой) = 0,8.*

Тогда в предположении независимости вероятность появления обоих слов в плохом документе

*Pr(Python & Casino | Плохой) = 0,8 × 0,2 = 0,16.*

Стало быть, вычисление вероятности для документа в целом сводится к перемножению вероятностей отдельных встречающихся в нем слов.

Создадим подкласс naivebayes класса classifier и включим в него метод docprob, который выделяет признаки (слова) и перемножает их вероятности:


In [50]:
class naivebayes(classifier):
    
    def docprob(self,item,cat):
        features=self.getfeatures(item)   

    # Multiply the probabilities of all the features together
        p=1
        for f in features: p*=self.weightedprob(f,cat,self.fprob)
        return p

Теперь вы знаете, как вычислить вероятность Pr(Документ | Категория), но само по себе это не слишком полезно. Для классификации документов нужно знать, чему равно значение Pr(Категория | Документ).

Иными словами, при условии наличия конкретного документа какова вероятность, что он попадает в данную категорию? К счастью, английский математик Томас Байес еще 250 лет назад понял, как это сделать

## Краткое введение в теорему Байеса
Теорема Байеса описывает соотношение между условными вероятностями. Обычно она записывается в виде

*Pr(A | B) = Pr(B | A) × Pr(A) / Pr(B)*

В применении к нашему примеру эта формула принимает следующий вид:

*Pr(Категория | Документ) = Pr(Документ | Категория) × Pr(Категория) / Pr(Документ)*

В предыдущем разделе мы показали, как вычислить Pr(Документ | Категория), но как быть с остальными двумя членами? Ничего сложного.

Pr(Категория) – это вероятность попадания случайно выбранного документа в данную категорию, поэтому она просто равна числу документов из этой категории, поделенному на общее число документов.

Что касается вероятности Pr(Документ), то ее тоже можно было бы вычислить, но это лишний труд. Напомним, что конечный результат не  будет использоваться как истинная вероятность. Мы лишь по отдельности вычислим вероятности попадания в каждую категорию, а затем сравним их. Поскольку величина Pr(Документ) одна и та же для всех категорий, то ее можно попросту игнорировать.

Метод prob вычисляет вероятность попадания в категорию и возвращает произведение Pr(Документ | Категория) и Pr(Категория). Добавим его в класс naivebayes:

In [51]:
class naivebayes(classifier):
        
    def docprob(self,item,cat):
        features=self.getfeatures(item)   

    # Multiply the probabilities of all the features together
        p=1
        for f in features: p*=self.weightedprob(f,cat,self.fprob)
        return p
  
    def prob(self,item,cat):
        catprob=self.catcount(cat)/self.totalcount()
        docprob=self.docprob(item,cat)
        return docprob*catprob


Протестируем эту функцию и посмотрим, как изменяются числа для разных строк и категорий:


In [52]:
cl=naivebayes(getwords)
sampletrain(cl)
print(cl.prob('quick rabbit','good'))
print(cl.prob('quick rabbit','bad'))


0.15624999999999997
0.05


По результатам обучения оказывается следующее: вероятность того, что фраза quick rabbit является частью хорошего документа, гораздо выше, чем плохого.


## Выбор категории
Последним шагом построения наивного байесовского классификатора является выбор категории, к которой следует отнести новый образец. Простейший подход состоит в том, чтобы вычислить вероятности попадания образца в каждую категорию и выбрать ту, для которой вероятность максимальна. Если вам просто нужно определить самое подходящее место, куда положить образец, то такая стратегия годится, но во многих приложениях категории нельзя считать равноценными, а иногда лучше, чтобы классификатор признал, что не знает точного ответа, и не пытался выдать за ответ категорию, для которой вероятность оказалась лишь чуть выше, чем для прочих.

В случае фильтрации спама гораздо важнее избежать классификации хороших сообщений как спама, чем отловить все спамные сообщения
без исключения. С редкими появлениями спамных сообщений в своем почтовом ящике вы готовы будете смириться, а вот помещение важного письма в ящик «Сомнительные» легко может остаться незамеченным. Если же вы внимательно просматриваете этот ящик в поисках важных сообщений, то к чему тогда вообще антиспамный фильтр?

Для решения этой проблемы можно задать для каждой категории минимальное пороговое значение. Чтобы новый образец был отнесен к некоторой категории, его вероятность должна быть по крайней мере на эту величину больше вероятности попадания в любую другую категорию. Эта величина называется порогом. Для фильтрации спама порог отнесения к категории плохих документов можно было бы установить равным 3. Это означает, что вероятность оказаться хорошим документом должна быть хотя бы в 3 раза выше, чем плохим. Для категории «Хороший» задается порог 1, то есть документ будет признаваться хорошим, если вероятность этого хоть чуть-чуть выше, чем для категории «Плохой». Все сообщения, для которых вероятность оказаться «плохим» выше, чем «хорошим», но менее чем в 3 раза, классифицируются как «Неизвестный».

Для задания порогов добавим в класс classifier новую переменную экземпляра, изменив метод инициализации:

In [53]:
class naivebayes(classifier):
  
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.thresholds={}
        
    def docprob(self,item,cat):
        features=self.getfeatures(item)   

    # Multiply the probabilities of all the features together
        p=1
        for f in features: p*=self.weightedprob(f,cat,self.fprob)
        return p
  
    def prob(self,item,cat):
        catprob=self.catcount(cat)/self.totalcount()
        docprob=self.docprob(item,cat)
        return docprob*catprob

Добавьте два простых метода для установки и получения значений, причем по умолчанию пусть возвращается значение 1,0:


In [54]:
class naivebayes(classifier):
  
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.thresholds={}
        
    def docprob(self,item,cat):
        features=self.getfeatures(item)   

    # Multiply the probabilities of all the features together
        p=1
        for f in features: p*=self.weightedprob(f,cat,self.fprob)
        return p
  
    def prob(self,item,cat):
        catprob=self.catcount(cat)/self.totalcount()
        docprob=self.docprob(item,cat)
        return docprob*catprob
    
    def setthreshold(self,cat,t):
        self.thresholds[cat]=t
    
    def getthreshold(self,cat):
        if cat not in self.thresholds: return 1.0
        return self.thresholds[cat]


Теперь все готово для написания метода classify. Он вычисляет вероятность для каждой категории, ищет среди них максимальную и проверяет, отличается ли она от следующей по порядку более чем в пороговоечисло раз. Если такую категорию найти не удается, то метод просто возвращает значение по умолчанию. Добавим в класс classifier такой код

In [55]:
import time
class naivebayes(classifier):
  
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.thresholds={}
    
    def docprob(self,item,cat):
        features=self.getfeatures(item)   
        p=1
        for f in features: p*=self.weightedprob(f,cat,self.fprob)
        return p
    def prob(self,item,cat):
        catprob=self.catcount(cat)/self.totalcount()
        docprob=self.docprob(item,cat)
        return docprob*catprob

  
    def setthreshold(self,cat,t):
        self.thresholds[cat]=t
    
    def getthreshold(self,cat):
        if cat not in self.thresholds: return 1.0
        return self.thresholds[cat]
  
    def classify (self, item, default = None):
        probs={}
        
        max = 0
        for cat in self.categories():
            probs[cat]=self.prob(item,cat)
            if probs[cat]>max: 
                max=probs[cat]
                best = cat

        # Make sure the probability exceeds threshold*next best
        for cat in probs:
            if cat==best: 
                continue
            
            if probs[cat]*self.getthreshold(best)>probs[best]: 
                return default
        return best


Все сделано! Вы построили законченную систему классификации документов. Ее можно обобщить на классификацию других образцов, написав методы для выделения признаков. Протестируйте свой классификатор в интерактивном сеансе:

In [56]:
сl6=naivebayes(getwords)

sampletrain(сl6);

print(сl6.classify('quick rabbit',default='unknown'))

print(сl6.classify('quick money',default='unknown'))

сl6.setthreshold('bad',3.0)
print(сl6.classify('quick money',default='unknown'))

for i in range(10):
    sampletrain(сl6);

print(сl6.classify('quick money',default='unknown'))


good
bad
unknown
bad


## Метод Фишера
Метод Фишера, названный по имени Р. А. Фишера (R. A. Fisher), – этоальтернативный метод классификации, который дает очень точные результаты, особенно применительно к фильтрации спама. Он используется в подключаемом к программе Outlook фильтре SpamBayes, который написан на языке Python. В отличие от наивной байесовской фильтрации, когда для вычисления вероятности всего документа перемножаются вероятности отдельных признаков, по методу Фишера вычисляетсявероятность отнесения к той или иной категории для каждого признака  документа, после чего эти вероятности комбинируются и проверяется, насколько получившееся множество похоже на случайное. Хотя этот метод более сложен, с ним стоит ознакомиться, так как он обеспечивает большую гибкость при настройке параметров классификации.

## Вероятности отнесения признаков к категориям
В наивном байесовском фильтре, который обсуждался выше, мы комбинировали вероятности Pr(Признак| Категория) для получения полной вероятности документа, а затем сравнивали их. В этом разделе мы начнем с вычисления вероятности попадания документа в некоторую категорию при условии наличия у этого документа конкретного признака, то есть с вычисления величины Pr(Категория | Признак).
Если слово casino встречается в 500 документах, из которых 499 плохие, то оценка casino относительно категории «Плохой» будет очень близка к 1.

Обычно значение Pr(Категория | Признак) вычисляется так:
(Количество документов с данным признаком в этой категории) / (Общее количество документов с данным признаком)

Эта формула не учитывает, что для одной категории могло быть предъявлено гораздо больше документов, чем для другой. Если у вас есть много хороших документов и совсем мало плохих, то у слова, встречающегося во всех плохих документах, будет высокая вероятность оказаться плохим, пусть даже сообщение с равным успехом могло бы быть и хорошим. Методы работают лучше в предположении, что в будущем будет получено равное количество документов из каждой категории,
поскольку это позволяет в полной мере воспользоваться признаками, по которым различаются категории.

Чтобы выполнить такую нормализацию, в методе Фишера вычисляются три величины:

clf = Pr(Признак | Категория) для этой категории

freqsum = сумма Pr(Признак | Категория) для всех категорий

cprob = clf / (clf + nclf)

Создадим новый подкласс fisherclassifier класса classifier и включите в него такой метод:


In [57]:
class fisherclassifier(classifier):
    def cprob(self,f,cat):
        # Частота появления данного признака в данной категории
        clf=self.fprob(f,cat)
        if clf==0: return 0
        # Частота появления данного признака во всех категориях
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        # Вероятность равна частоте появления в данной категории, поделенной на
        # частоту появления во всех категориях
        p=clf/(freqsum)
        return p

Эта функция возвращает вероятность того, что образец с указанным признаком принадлежит указанной категории, в предположении, что
в каждой категории будет одинаковое число образцов. Можете посмотреть в интерактивном сеансе, какие получаются цифры:


In [58]:
cl7 = fisherclassifier(getwords);
sampletrain(cl7);
print(cl7.cprob('quick','good'))
print(cl7.cprob('money','bad'))

0.5714285714285715
1.0


Как видите, документы, содержащие слово casino с вероятностью 0,9, классифицируются как спам. Это соответствует данным обучения, но остается все та же проблема – недостоверность для редко встречающихся слов, когда вероятность может оказаться сильно завышенной.
Поэтому, как и раньше, лучше пользоваться взвешенной вероятностью, начав со значения 0,5 и постепенно улучшая точность по мере обучения класса:


In [59]:
cl7.weightedprob('money','bad',cl7.cprob)

0.75

## Комбинирование вероятностей
Теперь надо из вероятностей отдельных признаков получить полную вероятность. Теоретически их можно просто перемножить и сравнить
результат для данной категории с результатами для других категорий. Конечно, из-за того что признаки не являются независимыми, истинной вероятности мы так не получим, но все же этот алгоритм работает гораздо лучше байесовского классификатора, разработанного в предыдущем разделе. Метод Фишера дает существенно более точную оценку вероятности, что может оказаться крайне полезным в отчетах или при решении вопроса об уровне отсечки.
По методу Фишера все вероятности перемножаются, затем берется натуральный логарифм (math.log в Python) и результат умножается на –2.
Добавим следующий метод в класс fisherclassifier:

In [60]:
class fisherclassifier(classifier):
    def cprob(self,f,cat):
        # Частота появления данного признака в данной категории
        clf=self.fprob(f,cat)
        if clf==0: return 0
        # Частота появления данного признака во всех категориях
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        # Вероятность равна частоте появления в данной категории, поделенной на
        # частоту появления во всех категориях
        p=clf/(freqsum)
        return p
    def fisherprob(self,item,cat):
        # Переменожить все вероятности
        p=1
        features=self.getfeatures(item)
        for f in features:
            p*=(self.weightedprob(f,cat,self.cprob))
        # Взять натуральный логарифм и умножить на -2
        fscore=-2*math.log(p)
        # Для получения вероятности пользуемся обратной функцией хи-квадрат
        return self.invchi2(fscore,len(features)*2)


Фишер доказал, что если бы вероятности были независимы и случайны, то результат этого вычисления подчинялся бы распределению хиквадрат. Следует ожидать, что образец, не принадлежащий некоторой категории, будет содержать слова с различными вероятностями признаков для данной категории (их распределение было бы случайным), а в образце, принадлежащем категории, будет много признаков с высокими вероятностями. Подав результат вычисления по формуле Фишера на вход обратной функции хи-квадрат, мы получаем вероятность того, что случайный набор вероятностей вернет такое большое число.
Добавьте обратную функцию хи-квадрат в класс fisherclassifier:

In [61]:
class fisherclassifier(classifier):
    def cprob(self,f,cat):
        # Частота появления данного признака в данной категории
        clf=self.fprob(f,cat)
        if clf==0: return 0
        # Частота появления данного признака во всех категориях
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        # Вероятность равна частоте появления в данной категории, поделенной на
        # частоту появления во всех категориях
        p=clf/(freqsum)
        return p
    def fisherprob(self,item,cat):
        # Переменожить все вероятности
        p=1
        features=self.getfeatures(item)
        for f in features:
            p*=(self.weightedprob(f,cat,self.cprob))
        # Взять натуральный логарифм и умножить на -2
        fscore=-2*math.log(p)
        # Для получения вероятности пользуемся обратной функцией хи-квадрат
        return self.invchi2(fscore,len(features)*2)
    def invchi2(self,chi,df):
        m = chi / 2.0
        sum = term = math.exp(-m)

        for i in range(1, df//2):
            term *= m / i
        sum += term
        return min(sum, 1.0)

Протестируйте метод Фишера и посмотрите, как он оценивает различные строки:


In [62]:
cl8=fisherclassifier(getwords)
sampletrain(cl8)
print(cl8.cprob('quick','good'))

print(cl8.fisherprob('quick rabbit','good'))

print(cl8.fisherprob('quick rabbit','bad'))

0.5714285714285715
0.78013986588958
0.35633596283335256


Как видите, результаты всегда оказываются в диапазоне от 0 до 1. Сами по себе они являются хорошей оценкой того, насколько точно документ соответствует категории. Поэтому классификатор можно сделать более изощренным.

## Классификация образцов
Значения, возвращенные функцией fisherprob, можно использовать для классификации. Вместо того чтобы задавать мультипликативные пороги, как в байесовском фильтре, можно определить нижние границы для каждой классификации. Тогда классификатор вернет максимальное значение, укладывающееся в эти границы. Для антиспамного фильтра минимальную границу для категории «Плохой» можно задать достаточно высокой, например 0,6. А для категории «Хороший» можно задать гораздо меньшую границу, скажем 0,2. Это уменьшает шансы попадания в эту категорию хороших сообщений, но допускает «просачивание» небольшого числа спамных сообщений в ящик для входящей почты. Любое сообщение, для которого оценка для категории «Хороший» меньше 0,2, а для категории «Плохой» меньше 0,6, классифицируется как «Неизвестный».

Создайте в классе fisherclassifier метод init, инициализирующий еще одну переменную экземпляра для хранения отсечек:

In [63]:
class fisherclassifier(classifier):
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.minimums={}

    def cprob(self,f,cat):
        # Частота появления данного признака в данной категории
        clf=self.fprob(f,cat)
        if clf==0: return 0
        # Частота появления данного признака во всех категориях
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        # Вероятность равна частоте появления в данной категории, поделенной на
        # частоту появления во всех категориях
        p=clf/(freqsum)
        return p
    def fisherprob(self,item,cat):
        # Переменожить все вероятности
        p=1
        features=self.getfeatures(item)
        for f in features:
            p*=(self.weightedprob(f,cat,self.cprob))
        # Взять натуральный логарифм и умножить на -2
        fscore=-2*math.log(p)
        # Для получения вероятности пользуемся обратной функцией хи-квадрат
        return self.invchi2(fscore,len(features)*2)
    def invchi2(self,chi,df):
        m = chi / 2.0
        sum = term = math.exp(-m)

        for i in range(1, df//2):
            term *= m / i
        sum += term
        return min(sum, 1.0)

Добавьте методы получения и установки значений, считая, что по умолчанию принимается значение 0:

In [64]:
class fisherclassifier(classifier):
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.minimums={}

    def cprob(self,f,cat):
        # Частота появления данного признака в данной категории
        clf=self.fprob(f,cat)
        if clf==0: return 0
        # Частота появления данного признака во всех категориях
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        # Вероятность равна частоте появления в данной категории, поделенной на
        # частоту появления во всех категориях
        p=clf/(freqsum)
        return p
    def fisherprob(self,item,cat):
        # Переменожить все вероятности
        p=1
        features=self.getfeatures(item)
        for f in features:
            p*=(self.weightedprob(f,cat,self.cprob))
        # Взять натуральный логарифм и умножить на -2
        fscore=-2*math.log(p)
        # Для получения вероятности пользуемся обратной функцией хи-квадрат
        return self.invchi2(fscore,len(features)*2)
    def invchi2(self,chi,df):
        m = chi / 2.0
        sum = term = math.exp(-m)

        for i in range(1, df//2):
            term *= m / i
        sum += term
        return min(sum, 1.0)
    def setminimum(self,cat,min):
        self.minimums[cat]=min
    def getminimum(self,cat):
        if cat not in self.minimums: return 0
        return self.minimums[cat]


И наконец добавим метод вычисления вероятностей для каждой категории и определения наилучшего результата, превышающего заданный минимум:

In [65]:
class fisherclassifier(classifier):
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.minimums={}

    def cprob(self,f,cat):
        # Частота появления данного признака в данной категории
        clf=self.fprob(f,cat)
        if clf==0: return 0
        # Частота появления данного признака во всех категориях
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        # Вероятность равна частоте появления в данной категории, поделенной на
        # частоту появления во всех категориях
        p=clf/(freqsum)
        return p
    def fisherprob(self,item,cat):
        # Переменожить все вероятности
        p=1
        features=self.getfeatures(item)
        for f in features:
            p*=(self.weightedprob(f,cat,self.cprob))
        # Взять натуральный логарифм и умножить на -2
        fscore=-2*math.log(p)
        # Для получения вероятности пользуемся обратной функцией хи-квадрат
        return self.invchi2(fscore,len(features)*2)
    def invchi2(self,chi,df):
        m = chi / 2.0
        sum = term = math.exp(-m)

        for i in range(1, df//2):
            term *= m / i
        sum += term
        return min(sum, 1.0)
    def setminimum(self,cat,min):
        self.minimums[cat]=min
    def getminimum(self,cat):
        if cat not in self.minimums: return 0
        return self.minimums[cat]
    def classify(self,item,default=None):
        # Цикл для поиска наилучшего результата
        best=default
        max=0.0
        for c in self.categories( ):
            p=self.fisherprob(item,c)
        # Проверяем, что значение больше минимума
            if p>self.getminimum(c) and p>max:
                best=c
                max=p
        return best

Испытаем классификатор по методу Фишера на тестовых данных. Введите в интерактивном сеансе следующие команды:



In [66]:
cl9=fisherclassifier(getwords)
sampletrain(cl9)
cl9.classify('quick rabbit')

print(cl9.classify('quick money'))

print(cl9.setminimum('bad',0.8))
print(cl9.classify('quick money'))

print(cl9.setminimum('good',0.4))
print(cl9.classify('quick money'))

bad
None
good
None
good


Результаты аналогичны полученным от наивного байесовского классификатора. Но считается, что на практике классификатор Фишера фильтрует спам лучше; правда, убедиться в этом на небольшом обучающем наборе вряд ли получится. Какой классификатор использовать – зависит от приложения. Заранее не скажешь, что будет работать лучше и какие выбирать отсечки. Впрочем, приведенный код позволяет поэкспериментировать с обоими алгоритмами, задавая разные параметры.

## Сохранение обученных классификаторов
В реальном приложении маловероятно, что обучение и классификацию удастся полностью провести в рамках одного сеанса. Если классификатор – часть веб-приложения, то, наверное, вы захотите сохранить результаты обучения, проведенного пользователем, и восстановить их, когда пользователь придет в следующий раз.
### Использование SQLite
В этом разделе мы покажем, как сохранить результаты обучения классификатора в базе данных, в данном случае SQLite. Если у вашего приложения много пользователей, одновременно обучающих и опрашивающих классификатор, то, вероятно, имеет смысл хранить счетчики
в базе. 

In [67]:
!pip install pysqlite3;

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [68]:
from sqlite3 import dbapi2 as sqlite

Код, приведенный в этом разделе, предназначен для замены словарей,
которые сейчас имеются в классе classifier, постоянным хранилищем
данных. Добавим в класс classifier метод *setdb* для открытия базы данных
и создания необходимых таблиц. Структура таблиц повторяет структуру заменяемых ими словарей.

Если вы собираетесь адаптировать классификатор для другой базы
данных, то, возможно, потребуется изменить предложения create table
в соответствии с диалектом SQL в используемой СУБД.

Необходимо заменить вспомогательные методы получения и увеличения счетчиков, также необходимо заменить методы для получения списка всех категорий и общего числа документов.

Также нужно добавить вызов commit в метод *train*, чтобы
сохранить обновленные счетчики.

In [69]:
class classifier:
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        self.fc={}
        # Counts of documents in each category
        self.cc={}
        self.getfeatures=getfeatures
        
    def setdb(self,dbfile):
        self.con=sqlite.connect(dbfile)    
        self.con.execute('create table if not exists fc(feature,category,count)')
        self.con.execute('create table if not exists cc(category,count)')


    def incf(self,f,cat):
        count=self.fcount(f,cat)
        if count==0:
            self.con.execute("insert into fc values ('%s','%s',1)" % (f.replace("'",'').replace('"','').replace('<b>','').replace('</b>',''),cat))
        else:
            self.con.execute("update fc set count=%d where feature='%s' and category='%s'" % (count+1,f.replace("'",'').replace('"','').replace('<b>','').replace('</b>',''),cat)) 

    def fcount(self,f,cat):
        res=self.con.execute('select count from fc where feature="%s" and category="%s"' %(f.replace("'",'').replace('"','').replace('<b>','').replace('</b>',''),cat)).fetchone()
        if res==None: return 0
        else: return float(res[0])

    def incc(self,cat):
        count=self.catcount(cat)
        if count==0:
            self.con.execute("insert into cc values ('%s',1)" % (cat))
        else:
            self.con.execute("update cc set count=%d where category='%s'" % (count+1,cat))    

    def catcount(self,cat):
        res=self.con.execute('select count from cc where category="%s"' %(cat)).fetchone()
        if res==None: return 0
        else: return float(res[0])

    def categories(self):
        cur=self.con.execute('select category from cc');
        return [d[0] for d in cur]

    def totalcount(self):
        res=self.con.execute('select sum(count) from cc').fetchone();
        if res==None: return 0
        return res[0]

    
    def train(self,item,cat):  
        features=self.getfeatures(item)
        # Увеличить счетчики для каждого признака в данной классификации
        for f in features:  
            self.incf(f,cat)  
            # Увеличить счетчик применений этой классификации  
        self.incc(cat)
        self.con.commit()
        
        
    def fprob(self,f,cat):  
        if self.catcount(cat)==0: return 0  
        # Общее число раз, когда данный признак появлялся в этой категории,  
        # делим на количество образцов в той же категории  
        return self.fcount(f,cat)/self.catcount(cat)   
    
    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        # Calculate current probability
        basicprob=prf(f,cat)

        # Count the number of times this feature has appeared in
        # all categories
        totals=sum([self.fcount(f,c) for c in self.categories()])

        # Calculate the weighted average
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp

Обновим классификаторы, чтобы добавились методы, которые мы изменили

In [70]:
class fisherclassifier(classifier):
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.minimums={}

    def cprob(self,f,cat):
        # Частота появления данного признака в данной категории
        clf=self.fprob(f,cat)
        if clf==0: return 0
        # Частота появления данного признака во всех категориях
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        # Вероятность равна частоте появления в данной категории, поделенной на
        # частоту появления во всех категориях
        p=clf/(freqsum)
        return p
    def fisherprob(self,item,cat):
        # Переменожить все вероятности
        p=1
        features=self.getfeatures(item)
        for f in features:
            p*=(self.weightedprob(f,cat,self.cprob))
        # Взять натуральный логарифм и умножить на -2
        fscore=-2*math.log(p)
        # Для получения вероятности пользуемся обратной функцией хи-квадрат
        return self.invchi2(fscore,len(features)*2)
    def invchi2(self,chi,df):
        m = chi / 2.0
        sum = term = math.exp(-m)

        for i in range(1, df//2):
            term *= m / i
        sum += term
        return min(sum, 1.0)
    def setminimum(self,cat,min):
        self.minimums[cat]=min
    def getminimum(self,cat):
        if cat not in self.minimums: return 0
        return self.minimums[cat]
    def classify(self,item,default=None):
        # Цикл для поиска наилучшего результата
        best=default
        max=0.0
        for c in self.categories( ):
            p=self.fisherprob(item,c)
        # Проверяем, что значение больше минимума
            if p>self.getminimum(c) and p>max:
                best=c
                max=p
        return best

In [71]:
class naivebayes(classifier):
  
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.thresholds={}
    
    def docprob(self,item,cat):
        features=self.getfeatures(item)   
        p=1
        for f in features: p*=self.weightedprob(f,cat,self.fprob)
        return p
    def prob(self,item,cat):
        catprob=self.catcount(cat)/self.totalcount()
        docprob=self.docprob(item,cat)
        return docprob*catprob

  
    def setthreshold(self,cat,t):
        self.thresholds[cat]=t
    
    def getthreshold(self,cat):
        if cat not in self.thresholds: return 1.0
        return self.thresholds[cat]
  
    def classify (self, item, default = None):
        probs={}
        
        max = 0
        for cat in self.categories():
            probs[cat]=self.prob(item,cat)
            if probs[cat]>max: 
                max=probs[cat]
                best = cat

        # Make sure the probability exceeds threshold*next best
        for cat in probs:
            if cat==best: 
                continue
            
            if probs[cat]*self.getthreshold(best)>probs[best]: 
                return default
        return best

In [72]:
cl11 = fisherclassifier(getwords)
cl11.setdb('test1.db')
sampletrain(cl11)

In [73]:
cl12 = naivebayes(getwords)
cl12.setdb('test2.db')
sampletrain(cl12)

In [74]:
cl12.classify('quick money')

'bad'

## Фильтрация блогов
Для тестирования классификатора на реальных данных и демонстрации различных способов использования можно применить его к записям из блогов или других RSS-каналов. Для этого потребуется библиотека Universal Feed Parser


In [75]:
!pip install feedparser

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Хотя в записях блога спама может и не быть, но многие блоги содержат
как интересующие вас заметки, так и не представляющие никакого
интереса. Например, иногда вы хотите читать только материалы из какой-то одной категории или написанные определенным автором, хотя
часто ситуация бывает и сложнее. Как и раньше, можно задать правила, описывающие, что именно представляет для вас интерес, а что
нет, – например, при чтении блога, посвященного электронным устройствам, вы хотите пропускать все записи, содержащие фразу cell
phone (сотовый телефон). Но гораздо проще воспользоваться классификатором, который сам выработает такие правила.
Классификация записей в RSS-каналах удобна тем, что можно воспользоваться каким-нибудь инструментом поиска по блогам, например Google Blog Search, и подготовить результаты поиска для подачи
на вход программе чтения канала. Многие делают так для отслеживания товаров, других интересных для них вещей и даже собственных
имен. Однако при этом вы будете находить заспамленные или бесполезные блоги, создатели которых пытаются сделать деньги на трафике.

Мы рассмотрим результаты поиска по слову Python, возвращенных Google Blog Search и представленных в формате RSS, хранящиеся в файле python_search.xml

In [76]:
import feedparser
import re

# Takes a filename of URL of a blog feed and classifies the entries
def read(feed,classifier):
    # Get feed entries and loop over them
    f=feedparser.parse(feed)
    for entry in f['entries']:
        print('')
        print ('-----')
        # Print the contents of the entry
        print ('Title:     '+ entry['title'])
        print ('Publisher: '+entry['publisher'])
        print('')
        print (entry['summary'])


        # Combine all the text to create one item for the classifier
        fulltext='%s\n%s\n%s' % (entry['title'],entry['publisher'],entry['summary'])

        # Print the best guess at the current category
        print ('Guess: '+str(classifier.classify(fulltext)))

        # Ask the user to specify the correct category and train on that
        cl=input('Enter category: ')
        classifier.train(fulltext,cl)



Эта функция перебирает все записи и с помощью классификатора вырабатывает наилучшую гипотезу о принадлежности к той или иной категории. Она сообщает об этой гипотезе пользователю и просит ввести
правильную категорию. Поначалу классификатор вырабатывает гипотезы случайным образом, но со временем его решения должны улучшаться.

Построенный классификатор по природе своей весьма общий. Мы воспользовались в качестве примера фильтрацией спама, чтобы объяснить все детали кода, но, в принципе, категории могут быть любыми.
Так, в файле python_search.xml можно выделить четыре категории:
о языке программирования, о комедийном шоу «Monty Python», о питонах и обо всем остальном.

In [77]:
cl22 = fisherclassifier(getwords)
cl22.setdb('python_feed.db') 

Попробуем обучить классификатор:

In [78]:
read('python_search.xml',cl22)


-----
Title:     My new baby boy!
Publisher: Shetan Noir, the zombie belly dancer! - MySpace Blog

THis is my new baby, Anthem. He is a 3 and half month old ball <b>python</b>, orange shaded normal pattern. I have held him about 5 time since I brought him home tonight at 8:00pm...
Guess: 3


Enter category:  snake



-----
Title:     If you need a laugh...
Publisher: Kate&#39;s space

Even does funny walks; from Monty <b>Python</b>. He talks about all the ol' ladies that are after him. He teases me about my horror obsession. He attempts suicide. And best of all, he talks about poo. Who doesn't think poo is funny???!
Guess: snake


Enter category:  monty



-----
Title:     And another one checked off the list..New pix comment ppl
Publisher: Python Guru - MySpace Blog

Now the one of a kind NERD bred Carplot male is in our possesion. His name is Broken (not because he is sterile) lol But check out the pic and leave one bitches..............
Guess: snake


...

Enter category:  prog



-----
Title:     Welcome John Lam to MS
Publisher: A blog to the tune of James

This is exciting news as MS opens up the CRL for Dynamic Languages like <b>Python</b> and Ruby. Now I just have to find some free time to come up to speed on Ruby... John gives a better take on the move and...(read more)
Guess: prog


Enter category:  prog



-----
Title:     White Gold
Publisher: Stellblog

Meeting up with Mere et Pere for a spot of déjeuner at the weekend, we were greeted by waiters who's French-ness appears to have come straight out of a Monty <b>Python</b> sketch... "Bonjour! And 'ow are we today?" <b>...</b>
Guess: prog


Enter category:  prog


Можно увидеть, что со временем гипотезы улучшаются. Примеров о змеях
довольно мало, поэтому классификатор часто допускает для них ошибки.

Далее можно посмотреть вероятности для указанного признака – как вероятность слова при условии категории, так и вероятность категории при условии слова:

In [79]:
cl22.cprob('eric','monty')

0

In [80]:
cl22.cprob('<b>python</b>','snake')

0.25749199883619434

In [81]:
cl22.cprob('language','programming')

0

In [82]:
cl22.fprob('eric','monty')

0.0

### Усовершенствование алгоритма обнаружения признаков
Во всех рассмотренных до сих пор примерах функция создания списка
признаков просто разбивает текст на слова по символам, отличным от
букв и цифр. Кроме того, она преобразует слова в нижний регистр, поэтому пропадает возможность обнаружить чрезмерное количество
слов, написанных заглавными буквами. Ситуацию можно улучшить
несколькими способами:

-Не считая слова, записанные заглавными и строчными буквами, совершенно различными, признать наличие большого количества
«кричащих» слов признаком.<br>
-Использовать помимо отдельных слов еще и словосочетания.<br>
-Собирать дополнительную метаинформацию, например, о том, кто
отправил письмо или в какую категорию была помещена запись
блога, и помечать, что это именно метаданные.<br>
-Сохранять URL и числа неизменными.

Не забывайте, что недостаточно просто выбрать более специфичные
признаки. Чтобы классификатор мог воспользоваться признаком, последний должен встречаться во многих документах.

Класс classifier может принять в качестве getfeatures любую функцию
и применять ее к предъявляемым образцам. Он ожидает лишь, что функция вернет список или словарь выделенных из образца признаков.
В силу общности природы классификатора вы можете написать функцию, которая работает с более сложными типами, чем строки. Например, для классификации записей в блоге можно придумать функцию, которая принимает всю запись целиком, а не только извлеченный из
нее текст, и вставляет аннотации о том, откуда взялось каждое слово.

Можно также выбирать пары слов из тела текста и отдельные слова из
темы. По всей видимости, разбивать на части поле creator бессмысленно, так как сообщения от некоего «Джона Смита» ничего не скажут
о сообщениях какого-нибудь другого «Джона».

Добавим следующую функцию выделения признаков. 

In [83]:
def entryfeatures(entry):
    splitter=re.compile('\s')
    f={}

    # Extract the title words and annotate
    titlewords=[s.lower() for s in splitter.split(entry['title']) if len(s)>2 and len(s)<20]
    for w in titlewords: f['Title:'+w]=1

    # Extract the summary words
    summarywords=[s.lower() for s in splitter.split(entry['summary'])  if len(s)>2 and len(s)<20]

    # Count uppercase words
    uc=0
    for i in range(len(summarywords)):
        w=summarywords[i]
        f[w]=1
        if w.isupper(): uc+=1

    # Get word pairs in summary as features
        if i<len(summarywords)-1:
            twowords=' '.join(summarywords[i:i+1])
            f[twowords]=1

    # Keep creator and publisher whole
    f['Publisher:'+entry['publisher']]=1

    # UPPERCASE is a virtual word flagging too much shouting  
    if float(uc)/len(summarywords)>0.3: f['UPPERCASE']=1

    return f

Эта функция извлекает слова из заголовка и резюме так же, как написанная ранее функция getwords. Все слова из заголовка она рассматривает как отдельные признаки. Слова из резюме тоже помещаются
в список слов, но дополнительно признаками считаются пары соседних слов. Информация об авторе трактуется как неделимый признак.
Напоследок функция подсчитывает количество слов, написанных заглавными буквами. Если их доля больше 30% от общего числа слов, то
функция добавляет в список специальный признак UPPERCASE. В отличие от правила, которое говорит, что написанные заглавными буквами
слова означают нечто особенное, мы просто включаем еще один признак, который можно использовать для обучения классификатора.
В некоторых случаях классификатор может решить, что этот признак
бесполезен для распознавания категории документа.

Если вы хотите применить новую версию выделения признаков совместно с функцией
read, то последнюю придется изменить, так чтобы она передавала
записи непосредственно классификатору, а не функции fulltext. Достаточно изменить функцию так:

In [84]:
# Takes a filename of URL of a blog feed and classifies the entries
def read(feed,classifier):
    # Get feed entries and loop over them
    f=feedparser.parse(feed)
    for entry in f['entries']:
        print('')
        print ('-----')
        # Print the contents of the entry
        print ('Title:     '+ entry['title'])
        print ('Publisher: '+entry['publisher'])
        print('')
        print (entry['summary'])


        # Combine all the text to create one item for the classifier
        fulltext='%s\n%s\n%s' % (entry['title'],entry['publisher'],entry['summary'])

        # Print the best guess at the current category
        print ('Guess: '+str(classifier.classify(entry)))

        # Ask the user to specify the correct category and train on that
        cl=input('Enter category: ')
        classifier.train(entry,cl)

In [85]:
cl31 = fisherclassifier(entryfeatures)
cl31.setdb('python_feed1.db')

In [86]:
read('python_search.xml',cl31)


-----
Title:     My new baby boy!
Publisher: Shetan Noir, the zombie belly dancer! - MySpace Blog

THis is my new baby, Anthem. He is a 3 and half month old ball <b>python</b>, orange shaded normal pattern. I have held him about 5 time since I brought him home tonight at 8:00pm...
Guess: None


Enter category:  prog



-----
Title:     If you need a laugh...
Publisher: Kate&#39;s space

Even does funny walks; from Monty <b>Python</b>. He talks about all the ol' ladies that are after him. He teases me about my horror obsession. He attempts suicide. And best of all, he talks about poo. Who doesn't think poo is funny???!
Guess: prog


Enter category:  prog



-----
Title:     And another one checked off the list..New pix comment ppl
Publisher: Python Guru - MySpace Blog

Now the one of a kind NERD bred Carplot male is in our possesion. His name is Broken (not because he is sterile) lol But check out the pic and leave one bitches..............
Guess: prog


...

Enter category:  prog



-----
Title:     Welcome John Lam to MS
Publisher: A blog to the tune of James

This is exciting news as MS opens up the CRL for Dynamic Languages like <b>Python</b> and Ruby. Now I just have to find some free time to come up to speed on Ruby... John gives a better take on the move and...(read more)
Guess: snake


Enter category:  prog



-----
Title:     White Gold
Publisher: Stellblog

Meeting up with Mere et Pere for a spot of déjeuner at the weekend, we were greeted by waiters who's French-ness appears to have come straight out of a Monty <b>Python</b> sketch... "Bonjour! And 'ow are we today?" <b>...</b>
Guess: snake


Enter category:  prog


С признаками можно еще много чего сделать. Построенный базовый
каркас позволяет определить функцию выделения признаков и настроить классификатор для работы с ней. Он будет классифицировать любой предъявленный образец при условии, что написанная функция
способна выделить из него признаки.
## Альтернативные методы
Оба рассмотренных в данной главе классификатора – это примеры *обучения с учителем*, когда программе предъявляются правильные результаты и она постепенно улучшает качество вырабатываемых гипотез. Рассмотренная в главе 4 искусственная нейронная сеть, которая
взвешивала результаты поиска на предмет их ранжирования, – еще
один пример обучения с учителем. Нейронную сеть можно было бы
адаптировать для решения задач из этой главы, если интерпретировать
признаки как входные сигналы, а выходные сигналы сопоставить всем
возможным классификациям. Аналогично, к задачам из этой главы
применим метод опорных векторов, который описывается в главе 9.

Для классификации документов используется именно байесовская
фильтрация просто потому, что она требует гораздо меньше вычислительных ресурсов, чем прочие методы. В почтовом сообщении могут
быть сотни и даже тысячи слов, а для простого обновления счетчиков
требуется куда меньше памяти и процессорного времени, чем для обучения нейронной сети такого размера. Как было показано, все это можно сделать в рамках базы данных. В зависимости от необходимой скорости обучения и получения ответа, а также от окружения, нейронная
сеть иногда может оказаться приемлемой альтернативой. Но сложность нейронной сети препятствует какому-либо взаимодействию
с ней. В этой главе вы видели, что вычисленные вероятности слов легкодоступны и известно, какой вклад они вносят в окончательную оценку. А для весов связей между нейронами сети нет столь же простой интерпретации.

С другой стороны, у нейронных сетей и машин опорных векторов есть
одно весомое преимущество над описанными выше классификаторами: они могут улавливать более сложные взаимосвязи между подаваемыми на вход признаками. В байесовском классификаторе у каждого
признака имеется вероятность принадлежности к каждой категории,
и для получения полной вероятности они комбинируются. В нейронной сети вероятность признака может изменяться в зависимости от наличия или отсутствия других признаков. Например, вы хотели бы блокировать спам от онлайновых казино, но интересуетесь ставками на
бегах; тогда слово casino следует считать плохим, если еще где-то в сообщении не встречается слово horse (лошадь). Наивный байесовский
классификатор не способен уловить такую перекрестную зависимость,
а нейронная сеть может.
